In [1]:
import sys
print (sys.version) #this should read as 3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]
                    #if not return to anacond environment installation steps

3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]


In [2]:
# this lets us use the figures interactively
%matplotlib notebook

import geopandas as gpd

import pandas as pd
from shapely.geometry import Point
from shapely.geometry import Polygon
import datetime

import cartopy.crs as ccrs
from cartopy.feature import ShapelyFeature

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

import rasterio as rio

import landsatxplore as landsatxplore

import json

import numpy as np 

In [ ]:
#generate matplotlib handles to create a legend of the features we put in our map.
def generate_handles(labels, colors, edge='k', alpha=1):
    lc = len(colors)  # get the length of the color list
    handles = []
    for i in range(len(labels)):
        handles.append(mpatches.Rectangle((0, 0), 1, 1, facecolor=colors[i % lc], edgecolor=edge, alpha=alpha))
    return handles

# create a scale bar of length 20 km in the upper right corner of the map
def scale_bar(ax, location=(0.92, 0.95)):
    llx0, llx1, lly0, lly1 = ax.get_extent(ccrs.PlateCarree())
    sbllx = (llx1 + llx0) / 2
    sblly = lly0 + (lly1 - lly0) * location[1]

    tmc = ccrs.TransverseMercator(sbllx, sblly)
    x0, x1, y0, y1 = ax.get_extent(tmc)
    sbx = x0 + (x1 - x0) * location[0]
    sby = y0 + (y1 - y0) * location[1]

    plt.plot([sbx, sbx - 20000], [sby, sby], color='k', linewidth=9, transform=tmc)
    plt.plot([sbx, sbx - 10000], [sby, sby], color='k', linewidth=6, transform=tmc)
    plt.plot([sbx-10000, sbx - 20000], [sby, sby], color='w', linewidth=6, transform=tmc)

    plt.text(sbx, sby-4500, '20 km', transform=tmc, fontsize=8)
    plt.text(sbx-12500, sby-4500, '10 km', transform=tmc, fontsize=8)
    plt.text(sbx-24500, sby-4500, '0 km', transform=tmc, fontsize=8)

# load the outline of Northern Ireland for a backdrop
#coral1outline = gpd.read_file('data/coral1.shp')
#coral2outline = gpd.read_file('data/coral2.shp')
#coral3outline = gpd.read_file('data/coral3.shp')
#coral4outline = gpd.read_file('data/coral4.shp')
#coral5outline = gpd.read_file('data/coral5.shp')
#coral6outline = gpd.read_file('data/coral6.shp')
plt.ion() #makes plot interactive

In [4]:
from landsatxplore.api import API
from landsatxplore.earthexplorer import EarthExplorer #help (landsatxplore.api)

In [5]:
help (landsatxplore.api)

Help on module landsatxplore.api in landsatxplore:

NAME
    landsatxplore.api - Python implementation of the Earth Explorer API.

CLASSES
    builtins.object
        API
    
    class API(builtins.object)
     |  EarthExplorer API.
     |  
     |  Methods defined here:
     |  
     |  __init__(self, username, password, version='1.4.1')
     |      EarthExplorer API.
     |  
     |  login(self, username, password)
     |      Get an API key.
     |  
     |  logout(self)
     |      Log out.
     |  
     |  lookup(self, dataset, id_list, inverse=False)
     |      Convert a list of legacy scene identifier to product identifiers.
     |      
     |      Parameters
     |      ----------
     |      dataset : str
     |          LANDSAT_TM_C1, LANDSAT_ETM_C1, or LANDSAT_8_C1.
     |      id_list : list of str
     |          List of scene id if inverse if False. List of product id if
     |          inverse is True.
     |      inverse : bool, optional
     |          If set to tru

In [6]:
api = API('janemaps','ulster202020') #log in

In [7]:
landsat5 = api.search(
    dataset='landsat_tm_c2_l1',
    latitude=20.39,
    longitude=-87.31,
    start_date='1984-03-01',
    end_date='2013-06-05',
    max_cloud_cover=5
)

landsat7 = api.search(
    dataset='landsat_etm_c2_l1',
    latitude=20.39,
    longitude=-87.31,
    start_date='1999-04-15',
    end_date='2013-02-11',
    max_cloud_cover=5
)

landsat8 = api.search(
    dataset='landsat_ot_c2_l1',
    latitude=20.39,
    longitude=-87.31,
    start_date='2013-02-11',
    end_date='2020-04-25',
    max_cloud_cover=5
)
    
    
print('{} landsat5 scenes found.'.format(len(landsat5)))

for scene in landsat5:
    print(scene['summary'])
    
print('{} landsat7 scenes found.'.format(len(landsat7)))

for rasterls7 in landsat7:
    print(rasterls7['displayId'])
    

print('{} landsat8 scenes found.'.format(len(landsat8)))

for scene in landsat8:
    print(scene['entityId'])

api.logout()

20 landsat5 scenes found.
ID: LT05_L1TP_018046_19840410_20200918_02_T1, Date Acquired: 1984/04/10, Path:  018, Row:  046
ID: LT05_L1TP_019046_19840417_20200918_02_T1, Date Acquired: 1984/04/17, Path:  019, Row:  046
ID: LT05_L1TP_019046_19850114_20200918_02_T1, Date Acquired: 1985/01/14, Path:  019, Row:  046
ID: LT05_L1TP_019046_19861203_20200917_02_T1, Date Acquired: 1986/12/03, Path:  019, Row:  046
ID: LT05_L1TP_019046_19870104_20201014_02_T1, Date Acquired: 1987/01/04, Path:  019, Row:  046
ID: LT05_L1TP_019046_19870426_20201014_02_T1, Date Acquired: 1987/04/26, Path:  019, Row:  046
ID: LT05_L1TP_019046_19870512_20201014_02_T1, Date Acquired: 1987/05/12, Path:  019, Row:  046
ID: LT04_L1TP_019046_19881216_20200917_02_T1, Date Acquired: 1988/12/16, Path:  019, Row:  046
ID: LT04_L1TP_019046_19890509_20200916_02_T1, Date Acquired: 1989/05/09, Path:  019, Row:  046
ID: LT05_L1TP_019046_19910216_20200915_02_T1, Date Acquired: 1991/02/16, Path:  019, Row:  046
ID: LT05_L1TP_019046_199

In [8]:
landsat5[0]['entityId'], landsat5[1]['entityId'], landsat5[2]['entityId'], landsat5[3]['entityId'], landsat5[4]['entityId'], landsat5[5]['entityId'], landsat5[6]['entityId'], landsat5[7]['entityId'], landsat5[8]['entityId'], landsat5[9]['entityId'], landsat5[10]['entityId'], landsat5[11]['entityId'], landsat5[12]['entityId'], landsat5[13]['entityId'], landsat5[14]['entityId'], landsat5[15]['entityId'], landsat5[16]['entityId'], landsat5[17]['entityId'], landsat5[18]['entityId'], landsat5[19]['entityId'], 

('LT50180461984101XXX03',
 'LT50190461984108XXX12',
 'LT50190461985014XXX07',
 'LT50190461986337XXX04',
 'LT50190461987004XXX01',
 'LT50190461987116XXX03',
 'LT50190461987132XXX02',
 'LT40190461988351XXX04',
 'LT40190461989129XXX01',
 'LT50190461991047XXX03',
 'LT50190461993052AAA04',
 'LT50190461993100AAA04',
 'LT50190461993324XXX04',
 'LT50180461993349XXX04',
 'LT50180461994064XXX02',
 'LT50190461995026XXX01',
 'LT50190461995106AAA01',
 'LT50180461995163AAA01',
 'LT50180461995243AAA02',
 'LT50190461995330AAA02')

In [9]:
landsat7[0]['entityId'], landsat7[1]['entityId'], landsat7[2]['entityId'], landsat7[3]['entityId'], landsat7[4]['entityId'], landsat7[5]['entityId'], landsat7[6]['entityId'], landsat7[7]['entityId'], landsat7[8]['entityId'], landsat7[9]['entityId'], landsat7[10]['entityId'], landsat7[11]['entityId'], landsat7[12]['entityId'], landsat7[13]['entityId'], landsat7[14]['entityId'], landsat7[15]['entityId'], landsat7[16]['entityId'], landsat7[17]['entityId'], landsat7[18]['entityId'], landsat7[19]['entityId'],

('LE70180462000025EDC00',
 'LE70190462000112EDC00',
 'LE70190462001066EDC00',
 'LE70180462001107PFS00',
 'LE70180462002094EDC00',
 'LE70180462003065EDC00',
 'LE70190462004347EDC00',
 'LE70180462005022ASN00',
 'LE70190462005301EDC00',
 'LE70180462006057EDC00',
 'LE70190462007339ASN00',
 'LE70180462008223EDC00',
 'LE70190462008310ASN00',
 'LE70190462008342ASN00',
 'LE70190462009024EDC00',
 'LE70180462009273EDC00',
 'LE70190462010059EDC00',
 'LE70190462010283EDC00',
 'LE70180462011007EDC00',
 'LE70180462011055EDC00')

In [10]:
landsat8[0]['entityId'], landsat8[1]['entityId'], landsat8[2]['entityId'], landsat8[3]['entityId'], landsat8[4]['entityId'], landsat8[5]['entityId'], landsat8[6]['entityId']

('LC80190462014046LGN01',
 'LC80180462014327LGN01',
 'LO80180462015042LGN01',
 'LC80180462016077LGN01',
 'LC80190462017022LGN01',
 'LC80190462019028LGN00',
 'LC80180462019101LGN00')

In [11]:
ee = EarthExplorer('janemaps', 'ulster202020')

ee.download(scene_id=landsat5[0]['entityId'],
 output_dir='assignment/data')

#ee = EarthExplorer('janemaps', 'ulster202020')

#ee.download(scene_id=landsat7[19]['entityId'],
 #output_dir='assignment/data')

#ee = EarthExplorer('janemaps', 'ulster202020')

#ee.download(scene_id=landsat8[6]['entityId'], 
 #output_dir='assignment/data')

ee.logout()

93.7MB [04:28, 365kB/s]                               


In [ ]:
#crop_area = pd.read_csv('data/area_of_interest.txt') #read the csv file from *txt format
#crop_area.head() #print head of file

dfcrop_area = pd.read_csv('data/area_of_interest.txt')

In [ ]:
dfcrop_area['geometry'] = list (zip(dfcrop_area['x'], dfcrop_area['y']))
dfcrop_area['geometry'] = dfcrop_area['geometry'].apply(Polygon)

In [ ]:
print (dir(crop_area))

#crop_area['geometry'] = list(zip(crop_area['x'], crop_area['y'])) 
#crop_area['geometry'] = crop_area['geometry'].apply(Polygon)  

In [ ]:
crop_area = pd.read_csv('data/area_of_interest.txt') #read the csv file from *txt format
crop_area.head() #print head of file

crop_area['geometry'] = list(zip(crop_area['x'], crop_area['y'])) 
crop_area['geometry'] = crop_area['geometry'].apply(Polygon)           


#crop_area['geometry'] = list(zip(crop_area['x'], crop_area['y'])) 
#crop_area['geometry'] = crop_area['geometry'].apply(Point)

gdf_crop_poly = gpd.GeoDataFrame(crop_area) # set crs project to EPSG4326 WGS84
gdf_crop_poly.set_crs("EPSG:4326", inplace=True)


In [ ]:
#Identified = gpd.read_file('data/Identified.shp')# import data files
#Unidentified = gpd.read_file('data/Unidentified.shp')

#print (Identified.head())# print counties table column titles
#print (Unidentified.head())

corala = gpd.read_file('data/corala.shp')
coralb = gpd.read_file('data/coralb.shp')
coralc = gpd.read_file('data/coralc.shp')

print (corala.head())
print (coralb.head())
print (coralc.head())

In [ ]:
#Identified.crs, Unidentified.crs, 
corala.crs, coralb.crs, coralc.crs #check crs projections for imported files, transform for compatibility as required

In [4]:
id_data = pd.read_csv('turtle/id_data.txt') #read the csv file from *txt format
noid_data = pd.read_csv('turtle/noid_data.txt')

id_data.head() #print head of file
noid_data.head()

id_data['geometry'] = list(zip(id_data['LONG'], id_data['LAT'])) # zip is an iterator, so we use list to create 
                                                                 # something that pandas can use.
id_data['geometry'] = id_data['geometry'].apply(Point)           # using the 'apply' method of the dataframe, 
                                                                 # turn the coordinates column
                                                                 # into points (instead of a tuple of lat, lon coordinates).
                                                                 # NB: Point takes (x, y) coordinates

noid_data['geometry'] = list(zip(id_data['LONG'], noid_data['LAT'])) 
noid_data['geometry'] = noid_data['geometry'].apply(Point)

gdfid_data = gpd.GeoDataFrame(id_data) # set crs project to EPSG4326 WGS84
gdfid_data.set_crs("EPSG:4326", inplace=True)

gdfnoid_data = gpd.GeoDataFrame(noid_data)
gdfnoid_data.set_crs("EPSG:4326", inplace=True)


,Number,Date,Time_code,Time,Name,ID_Type,Photo,LAT,LONG,S_F_R,Number.1,H_V,D_T_C,geometry
0,1,12/03/2009,1,09:20,NaN,NaN,NaN,20.394083,-87.313450,S,NaN,NaN,NaN,POINT (-87.31372 20.39408)
1,3,12/03/2009,1,09:25,NaN,NaN,NaN,20.394667,-87.313850,F,NaN,NaN,NaN,POINT (-87.31385 20.39467)
2,9,12/03/2009,1,09:42,NaN,NaN,NaN,20.394450,-87.314000,S,NaN,NaN,NaN,POINT (-87.31385 20.39445)
3,10,12/03/2009,2,13:22,NaN,NaN,NaN,20.395283,-87.313167,S,NaN,NaN,NaN,POINT (-87.31383 20.39528)
4,12,12/03/2009,2,13:52,NaN,NaN,NaN,20.394617,-87.314583,S,NaN,NaN,NaN,POINT (-87.31328 20.39462)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,583,11/06/2009,1,09:21,NaN,NaN,NaN,20.393600,-87.312150,S,NaN,NaN,NaN,POINT (-87.31372 20.39360)
110,646,18/06/2009,1,10:02,NaN,NaN,IMG_0016,20.395300,-87.314500,S,NaN,NaN,NaN,POINT (-87.31390 20.39530)
111,692,22/06/2009,2,13:02,NaN,L PROFILE,IMG_0706/7,20.394317,-87.313533,E,NaN,NaN,NaN,POINT (-87.31403 20.39432)
112,715,25/06/2009,1,10:00,NaN,NaN,IMG_1304,20.394167,-87.313583,F,NaN,NaN,NaN,POINT (-87.31375 20.39417)


In [ ]:
gdfid_data.to_file('id_data.shp') #make csv read file into shape creates and saves new .shp file
print('id_data.shp')

rows, cols = gdfid_data.shape #display total number of rows
print('id_data Number of indentified turtle sightings:{}'.format(rows))
rows, cols = gdfnoid_data.shape
print('noid_data Number of unidentified turtle sightings:{}'.format(rows))

In [ ]:
FRANKIE = gdfid_data.loc[gdfid_data['ID'] == ['AK090001']]
topic = "Geographic Information Systems"
topic.count("i")


In [ ]:
rows, cols = corala.shape # create GeoDataFrame
print('corala Number of GPS waypoints:{}'.format(rows)) 
rows, cols = coralb.shape
print('coralb Number of GPS waypoints:{}'.format(rows))
rows, cols = coralc.shape
print('coralc Number of GPS waypoints: {}'.format(rows))

In [ ]:
corala.columns, coralb.columns, coralc.columns #prints the column heads, like print(coral.head()) but without any data

In [ ]:
coralc['LAT']

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon

lon_point_corala = [-87.312917,- 87.312983, -87.313067, -87.313117, -87.313150, -87.313267, -87.313283, -87.313200, 
-87.313100, -87.313033, -87.312983, -87.312950, -87.312833, -87.312733, -87.312717, -87.312767, -87.312767, -87.312733, 
-87.312767, -87.312833, -87.312833, -87.312867, -87.312700, -87.312667, -87.312583, -87.312533, -87.312467, -87.312417,
-87.312433, -87.312450, -87.312467, -87.312417, -87.312383, -87.312367, -87.312283, -87.312250, -87.312267, -87.312217,
-87.312750, -87.312850, -87.312917, -87.312917, -87.312883, -87.312817, -87.312833, -87.312717, -87.312700, -87.313400,
-87.313233, -87.313133, -87.313000, -87.312900, -87.312883, -87.312900, -87.312950, -87.313283, -87.314083, -87.314067,
-87.314017, -87.314033, -87.314133, -87.314167, -87.314267, -87.314217, -87.314100, -87.314017, -87.314067, -87.314167,
-87.314283, -87.314400, -87.314433, -87.314533, -87.314567, -87.314617, -87.314633, -87.314650, -87.314717, -87.314717,
-87.314733, -87.314800, -87.314933, -87.315133, -87.315317, -87.315433, -87.315517, -87.315600, -87.315650, -87.315633,
-87.315567, -87.315467, -87.315400, -87.315300, -87.314967, -87.314850, -87.314733, -87.314583, -87.314517, -87.314433,
-87.314333, -87.314217, -87.314133]
lat_point_corala = [20.393917, 20.393917, 20.393933, 20.393850, 20.393800, 20.393800, 20.393733, 20.393683, 20.393583,
20.393517, 20.393450, 20.393333, 20.393333, 20.393383, 20.393450, 20.393500, 20.393567, 20.393667, 20.393717, 20.393700,
20.393783, 20.393817, 20.393400, 20.393417, 20.393400, 20.393433, 20.393433, 20.393450, 20.393533, 20.393600, 20.393633,
20.393633, 20.393567, 20.393483, 20.393517, 20.393583, 20.393633, 20.393667, 20.393350, 20.393300, 20.393283, 20.393233,
20.393133, 20.393083, 20.393033, 20.393017, 20.393067, 20.392967, 20.392850, 20.392833, 20.392800, 20.392600, 20.392667,
20.392783, 20.392850, 20.392950, 20.393000, 20.393083, 20.393233, 20.393333, 20.393400, 20.393417, 20.393350, 20.393633,
20.393700, 20.393717, 20.393783, 20.393783, 20.393783, 20.393800, 20.393867, 20.393917, 20.393850, 20.393750, 20.393667,
20.393650, 20.393667, 20.393600, 20.393550, 20.393500, 20.393433, 20.393450, 20.393483, 20.393467, 20.393450, 20.393483,
20.393400, 20.393350, 20.393250, 20.393217, 20.393167, 20.393067, 20.393150, 20.393167, 20.393133, 20.393150, 20.393133,
20.393117, 20.393067, 20.393067, 20.393033]

polygon_geom_corala = Polygon(zip(lon_point_corala, lat_point_corala))
crs = {'init': 'epsg:4326'}
corala_poly = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom_corala])       
print(corala_poly.geometry)



#C:\Users\jrllo\anaconda3\envs\turtlepython\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  #return _prepare_from_string(" ".join(pjargs))
               #need to fix this it works but is not a future proof technique


In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon

lon_point_coralb = [-87.313150, -87.313183, -87.313150, -87.313083, -87.313067, -87.313017, -87.312983, -87.313033,
-87.312950, -87.312933, -87.312950, -87.312900, -87.312883, -87.312950, -87.312983, -87.312883, -87.312850, -87.312783,
-87.312750, -87.312750, -87.312817, -87.312717, -87.312717, -87.312733, -87.312733, -87.312733, -87.312683, -87.312600,
-87.312533, -87.312500, -87.312533, -87.312550, -87.312483, -87.312433, -87.312400, -87.312367, -87.312350, -87.312417,
-87.312467, -87.312533, -87.312517, -87.312467, -87.312400, -87.312367, -87.312350, -87.312300, -87.312317, -87.312367,
-87.312433, -87.312467, -87.312433, -87.312417, -87.312417, -87.312450, -87.312500, -87.312517, -87.312550, -87.312600,
-87.312617, -87.312600, -87.312350, -87.312450, -87.312517, -87.312533, -87.312583, -87.312600, -87.312600, -87.312583, 
-87.312617, -87.312633, -87.312683, -87.312650, -87.312633, -87.312583, -87.312533, -87.312533, -87.312567, -87.312567,
-87.312500, -87.312483, -87.312450, -87.312433, -87.312417, -87.312400, -87.312383, -87.312500, -87.312417, -87.312367,
-87.312333, -87.312283, -87.312317, -87.312333, -87.312300, -87.312250, -87.312200, -87.312167, -87.312250, -87.312283,
-87.312317]
lat_point_coralb = [20.394367, 20.394383, 20.394383, 20.394383, 20.394417, 20.394450, 20.394433, 20.394367, 20.394433,
20.394450, 20.394467, 20.394533, 20.394550, 20.394600, 20.394633, 20.394600, 20.394600, 20.394600, 20.394617, 20.394550,
20.394517, 20.394467, 20.394483, 20.394500, 20.394483, 20.394600, 20.394683, 20.394767, 20.394767, 20.394817, 20.394883,
20.394933, 20.394967, 20.394917, 20.394850, 20.394800, 20.394750, 20.394717, 20.394717, 20.394733, 20.394700, 20.394600,
20.394483, 20.394417, 20.394317, 20.394217, 20.394150, 20.394150, 20.394150, 20.394167, 20.394217, 20.394233, 20.394300,
20.394333, 20.394367, 20.394400, 20.394383, 20.394383, 20.394433, 20.394483, 20.394067, 20.394117, 20.394150, 20.394150,
20.394100, 20.394067, 20.394033, 20.393967, 20.393983, 20.394017, 20.394000, 20.393933, 20.393900, 20.393767, 20.393700,
20.393683, 20.393650, 20.393617, 20.393633, 20.393650, 20.393683, 20.393733, 20.393850, 20.393883, 20.393967, 20.394100,
20.394100, 20.394133, 20.394150, 20.394217, 20.394300, 20.394417, 20.394450, 20.394500, 20.394533, 20.394533, 20.394583,
20.394550, 20.394517]

polygon_geom_coralb = Polygon(zip(lon_point_coralb, lat_point_coralb))
crs = {'init': 'epsg:4326'}
coralb_poly = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom_coralb])       
print(coralb_poly.geometry)


In [ ]:
from shapely.geometry import Polygon

lon_point_coralc = [-87.312917, -87.312867, -87.312800, -87.312800, -87.312967, -87.313000, -87.313033, -87.313050,
-87.313100, -87.313100, -87.313017, -87.312950, -87.312917, -87.312883, -87.312867, -87.312900, -87.312900, -87.312817,
-87.312817, -87.312850, -87.312767, -87.312633, -87.312600, -87.312567, -87.312517, -87.312533, -87.312567, -87.312533,
-87.312517, -87.312517, -87.312633, -87.312667, -87.312733, -87.312800, -87.312817, -87.312917, -87.313017, -87.313083,
-87.313167, -87.313233, -87.313283, -87.313233, -87.313217, -87.313300, -87.313200, -87.312800]
lat_point_coralc = [20.394583, 20.394617, 20.394633, 20.394667, 20.394667, 20.394783, 20.394833, 20.394900, 20.394883,
20.394850, 20.394883, 20.394883, 20.394933, 20.395017, 20.395100, 20.395183, 20.395317, 20.395383, 20.395433, 20.395483,
20.395467, 20.395367, 20.395333, 20.395183, 20.395050, 20.394950, 20.394900, 20.394833, 20.394850, 20.394917, 20.394900,
20.394867, 20.394800, 20.394767, 20.394617, 20.394583, 20.394550, 20.394467, 20.394433, 20.394433, 20.394467, 20.394583,
20.394750, 20.394450, 20.394650, 20.395534]

polygon_geom_coralc = Polygon(zip(lon_point_coralc, lat_point_coralc))
crs = {'init': 'epsg:4326'}
coralc_poly = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom_coralc])       
print(coralc_poly.geometry)

coralc_poly.to_file(filename='polygon.geojson', driver='GeoJSON') #coral_c_poly saved as shp file
coralc_poly.to_file(filename='coral_c_poly.shp', driver="ESRI Shapefile")

In [ ]:
#join = gpd.sjoin(corala, coralb, coralc how='inner', lsuffix='left', rsuffix='right') # perform the spatial join
#join # show the joined table

In [ ]:
raster = rio.open('data/akumal.tif')
akumal_raster = rio.open('data/akumal.tif')

print('{} opened in {} mode'.format(akumal_raster.name,akumal_raster.mode)) #name attribute is the filename for the dataset, and the mode refers to how the dataset has been opened (r for read, w for write, r+ for read/write).
print('image has {} band(s)'.format(akumal_raster.count)) #check how many layers, or bands, the datset has using count
print('image size (width, height): {} x {}'.format(akumal_raster.width, akumal_raster.height)) # check the size of the image using width and height
print('band 1 dataype is {}'.format(akumal_raster.dtypes[0])) # note that the band name (Band 1) differs from the list index [0]

In [ ]:
print(raster.bounds)

In [ ]:
print(raster.crs)

In [ ]:
img = akumal_raster.read(1)

In [ ]:
print(img.shape)
print(img[6])

In [ ]:
with rio.open('data/akumal.tif') as dataset:
    img = dataset.read()
    xmin, ymin, xmax, ymax = dataset.bounds

In [ ]:
myCRS = ccrs.UTM(16) #work out crs.utm() number from epsg value above
fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw=dict(projection=myCRS))

In [ ]:
ax.imshow(img[2], cmap='gray', vmin=200, vmax=5000)